In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import env

In [3]:
#read in data from sql
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)

In [4]:
df.head()

,entry
0,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
1,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
2,97.105.19.58 - - [16/Apr/2019:19:34:44 +0000] ...
3,97.105.19.58 - - [16/Apr/2019:19:34:46 +0000] ...
4,97.105.19.58 - - [16/Apr/2019:19:34:48 +0000] ...


In [5]:
df = df.entry.str.split(expand=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/sales?page=81,"HTTP/1.1""",200,512495,...,None,None,None,None,None,None,None,None,None,None
1,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/items,"HTTP/1.1""",200,3561,...,None,None,None,None,None,None,None,None,None,None
2,97.105.19.58,-,-,[16/Apr/2019:19:34:44,+0000],"""GET",/api/v1/sales?page=82,"HTTP/1.1""",200,510103,...,None,None,None,None,None,None,None,None,None,None
3,97.105.19.58,-,-,[16/Apr/2019:19:34:46,+0000],"""GET",/api/v1/sales?page=83,"HTTP/1.1""",200,510003,...,None,None,None,None,None,None,None,None,None,None
4,97.105.19.58,-,-,[16/Apr/2019:19:34:48,+0000],"""GET",/api/v1/sales?page=84,"HTTP/1.1""",200,511963,...,None,None,None,None,None,None,None,None,None,None


In [6]:
df[11] = df[11].str.strip('"')
df[11]

0        python-requests/2.21.0
1        python-requests/2.21.0
2        python-requests/2.21.0
3        python-requests/2.21.0
4        python-requests/2.21.0
                  ...          
13969    python-requests/2.21.0
13970    python-requests/2.21.0
13971    python-requests/2.21.0
13972    python-requests/2.21.0
13973    python-requests/2.21.0
Name: 11, Length: 13974, dtype: object

In [7]:
#combine using lambda
df[11] = df.loc[:,11:].apply(lambda rows: ' '.join(rows.values.astype('str'))
                    , axis=1).str.replace('None', '').str.strip()

In [8]:
#remove end columns using loc
df = df.loc[:,:11]
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/sales?page=81,"HTTP/1.1""",200,512495,"""-""",python-requests/2.21.0
1,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/items,"HTTP/1.1""",200,3561,"""-""",python-requests/2.21.0
2,97.105.19.58,-,-,[16/Apr/2019:19:34:44,+0000],"""GET",/api/v1/sales?page=82,"HTTP/1.1""",200,510103,"""-""",python-requests/2.21.0
3,97.105.19.58,-,-,[16/Apr/2019:19:34:46,+0000],"""GET",/api/v1/sales?page=83,"HTTP/1.1""",200,510003,"""-""",python-requests/2.21.0
4,97.105.19.58,-,-,[16/Apr/2019:19:34:48,+0000],"""GET",/api/v1/sales?page=84,"HTTP/1.1""",200,511963,"""-""",python-requests/2.21.0


In [9]:
#drop other unnecesary columns
df = df.drop(columns=[1,2,4,10])
df.head()

,0,3,5,6,7,8,9,11
0,97.105.19.58,[16/Apr/2019:19:34:42,"""GET",/api/v1/sales?page=81,"HTTP/1.1""",200,512495,python-requests/2.21.0
1,97.105.19.58,[16/Apr/2019:19:34:42,"""GET",/api/v1/items,"HTTP/1.1""",200,3561,python-requests/2.21.0
2,97.105.19.58,[16/Apr/2019:19:34:44,"""GET",/api/v1/sales?page=82,"HTTP/1.1""",200,510103,python-requests/2.21.0
3,97.105.19.58,[16/Apr/2019:19:34:46,"""GET",/api/v1/sales?page=83,"HTTP/1.1""",200,510003,python-requests/2.21.0
4,97.105.19.58,[16/Apr/2019:19:34:48,"""GET",/api/v1/sales?page=84,"HTTP/1.1""",200,511963,python-requests/2.21.0


In [10]:
#rename columns
df = df.rename(columns={0:'ip',
                  3:'timestamp',
                  5:'request_method',
                  6:'request_path',
                  7:'http_version',
                  8:'status_code',
                  9:'size',
                  11:'user_agent'})
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,[16/Apr/2019:19:34:42,"""GET",/api/v1/sales?page=81,"HTTP/1.1""",200,512495,python-requests/2.21.0
1,97.105.19.58,[16/Apr/2019:19:34:42,"""GET",/api/v1/items,"HTTP/1.1""",200,3561,python-requests/2.21.0
2,97.105.19.58,[16/Apr/2019:19:34:44,"""GET",/api/v1/sales?page=82,"HTTP/1.1""",200,510103,python-requests/2.21.0
3,97.105.19.58,[16/Apr/2019:19:34:46,"""GET",/api/v1/sales?page=83,"HTTP/1.1""",200,510003,python-requests/2.21.0
4,97.105.19.58,[16/Apr/2019:19:34:48,"""GET",/api/v1/sales?page=84,"HTTP/1.1""",200,511963,python-requests/2.21.0


In [11]:
#strip extra quotes
df.request_method = df.request_method.str.strip('"')
df.http_version = df.http_version.str.strip('"')
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,[16/Apr/2019:19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,[16/Apr/2019:19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,[16/Apr/2019:19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,[16/Apr/2019:19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,[16/Apr/2019:19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [12]:
df.timestamp = df.timestamp.str.strip('[').str.replace(':', ' ', 1)
df.timestamp = pd.to_datetime(df.timestamp)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,2019-04-16 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,2019-04-16 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,2019-04-16 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [13]:
# Synthetic data
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

new

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,95.31.18.119,21/Apr/2019 10:02:41,GET,/api/v1/items/,HTTP/1.1,200,1153005,python-requests/2.21.0
1,95.31.16.121,17/Apr/2019 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
2,97.105.15.120,18/Apr/2019 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0
3,97.105.19.58,19/Apr/2019 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,200,2056327,python-requests/2.21.0


In [14]:
df = pd.concat([df,new])
df = df.reset_index(drop=True)
df

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,2019-04-16 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,2019-04-16 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,2019-04-16 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0
...,...,...,...,...,...,...,...,...
13973,97.105.19.58,2019-04-17 12:55:14,GET,/api/v1/sales?page=71,HTTP/1.1,200,510166,python-requests/2.21.0
13974,95.31.18.119,21/Apr/2019 10:02:41,GET,/api/v1/items/,HTTP/1.1,200,1153005,python-requests/2.21.0
13975,95.31.16.121,17/Apr/2019 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
13976,97.105.15.120,18/Apr/2019 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13978 entries, 0 to 13977
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ip              13978 non-null  object
 1   timestamp       13978 non-null  object
 2   request_method  13978 non-null  object
 3   request_path    13978 non-null  object
 4   http_version    13978 non-null  object
 5   status_code     13978 non-null  object
 6   size            13978 non-null  object
 7   user_agent      13978 non-null  object
dtypes: object(8)
memory usage: 873.8+ KB


In [16]:
df.timestamp = df.timestamp.astype('datetime64')

In [17]:
df['size_mb'] = df['size'].astype(int) / 1024 / 1024
df = df.set_index('timestamp')
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb
timestamp,,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396
2019-04-16 19:34:44,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472
2019-04-16 19:34:46,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377
2019-04-16 19:34:48,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246


In [18]:
#save ip_counts into separate dataframe
ip_df = pd.DataFrame(df.ip.value_counts(dropna=False)).reset_index().\
                rename(columns={'index':'ip','ip':'count'})
ip_df

,ip,count
0,97.105.19.58,11999
1,173.173.113.51,1059
2,72.181.113.170,613
3,72.181.105.81,246
4,68.201.219.223,21
5,24.26.242.9,21
6,35.175.171.137,2
7,52.87.230.102,2
8,70.121.214.34,2
9,52.90.165.200,1


In [19]:
ip_df['prob'] = df.ip.value_counts(normalize=True).reset_index().ip

In [20]:
ip_df

,ip,count,prob
0,97.105.19.58,11999,0.858420
1,173.173.113.51,1059,0.075762
2,72.181.113.170,613,0.043855
3,72.181.105.81,246,0.017599
4,68.201.219.223,21,0.001502
5,24.26.242.9,21,0.001502
6,35.175.171.137,2,0.000143
7,52.87.230.102,2,0.000143
8,70.121.214.34,2,0.000143
9,52.90.165.200,1,0.000072


In [21]:
status_code_ip = df.groupby('ip').status_code.value_counts(normalize=True).\
                rename('proba_status_code_ip').reset_index()
status_code_ip.head()

,ip,status_code,proba_status_code_ip
0,173.173.113.51,200,1.0
1,24.26.242.9,200,1.0
2,3.88.129.158,200,1.0
3,3.92.201.136,200,1.0
4,34.207.64.242,200,1.0


In [22]:
df = df.merge(status_code_ip, how='left', on=['ip','status_code'])
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_status_code_ip
0,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753,0.998833
1,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396,0.998833
2,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472,0.998833
3,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377,0.998833
4,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246,0.998833


In [23]:
df.request_method.value_counts()

GET    13978
Name: request_method, dtype: int64

In [24]:
df.http_version.value_counts()

HTTP/1.1    13978
Name: http_version, dtype: int64

In [25]:
df.size_mb.value_counts()

0.471896    705
0.003396    548
0.003489    291
0.001266    229
0.001704    219
           ... 
0.327981      1
0.343606      1
0.312356      1
0.265481      1
1.961066      1
Name: size_mb, Length: 191, dtype: int64

In [26]:
df[df.proba_status_code_ip < .01]

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_status_code_ip
1745,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167
1819,97.105.19.58,GET,/api/v1/sales?page=21,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167
4242,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167
4405,97.105.19.58,GET,/api/v1/sales?page=29,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167
4790,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167
4862,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167
4871,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167
5520,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167
5539,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167
6860,97.105.19.58,GET,/api/v1/sales?page=115,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167


### We can see that these lower instaces of of 499 have a size of 0 and are attempting to access various different pages of sales and items

In [28]:
df[df.proba_status_code_ip < .01].user_agent.value_counts()

python-requests/2.21.0    12
python-requests/2.20.1     4
Name: user_agent, dtype: int64

### Two different user agent identifiers as well

In [32]:
df[df.status_code != '200']

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_status_code_ip
1745,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.20.1,0.000000,0.001167
1819,97.105.19.58,GET,/api/v1/sales?page=21,HTTP/1.1,499,0,python-requests/2.20.1,0.000000,0.001167
4242,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.000000,0.001167
4405,97.105.19.58,GET,/api/v1/sales?page=29,HTTP/1.1,499,0,python-requests/2.21.0,0.000000,0.001167
4790,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.000000,0.001167
4862,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.000000,0.001167
4871,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.000000,0.001167
5520,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.000000,0.001167
5539,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.000000,0.001167
6860,97.105.19.58,GET,/api/v1/sales?page=115,HTTP/1.1,499,0,python-requests/2.20.1,0.000000,0.001167


### Mostly 499 errors with 301 sprinkled in there

In [33]:
df.user_agent.value_counts()

python-requests/2.21.0                                                                                                        12005
python-requests/2.20.1                                                                                                         1911
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"       34
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"                                               8
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)"                                                                       7
Slackbot 1.0 (+https://api.slack.com/robots)"                                                                                     6
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"        4
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, l

In [37]:
user_df = pd.DataFrame(df.user_agent.value_counts(dropna=False)).reset_index().\
                rename(columns={'index':'user_agent','user_agent':'count'})
user_df

,user_agent,count
0,python-requests/2.21.0,12005
1,python-requests/2.20.1,1911
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,34
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,8
4,Slackbot-LinkExpanding 1.0 (+https://api.slack...,7
5,"Slackbot 1.0 (+https://api.slack.com/robots)""",6
6,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,4
7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,2
8,Python-urllib/3.7,1


In [38]:
user_df['prob'] = df.user_agent.value_counts(normalize=True).reset_index().user_agent
user_df

,user_agent,count,prob
0,python-requests/2.21.0,12005,0.858850
1,python-requests/2.20.1,1911,0.136715
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,34,0.002432
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,8,0.000572
4,Slackbot-LinkExpanding 1.0 (+https://api.slack...,7,0.000501
5,"Slackbot 1.0 (+https://api.slack.com/robots)""",6,0.000429
6,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,4,0.000286
7,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,2,0.000143
8,Python-urllib/3.7,1,0.000072


In [41]:
status_code_user = df.groupby('user_agent').status_code.value_counts(normalize=True).\
                rename('proba_status_code_user').reset_index()
status_code_user.head()

,user_agent,status_code,proba_status_code_user
0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,200,1.0
1,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,200,1.0
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,200,1.0
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,200,1.0
4,Python-urllib/3.7,200,1.0


In [42]:
df = df.merge(status_code_user, how='left', on=['user_agent','status_code'])
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_status_code_ip,proba_status_code_user
0,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753,0.998833,0.998834
1,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396,0.998833,0.998834
2,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472,0.998833,0.998834
3,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377,0.998833,0.998834
4,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246,0.998833,0.998834


In [46]:
df[(df.proba_status_code_user < .01) & (df.proba_status_code_ip < .01)]

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_status_code_ip,proba_status_code_user
1745,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167,0.002093
1819,97.105.19.58,GET,/api/v1/sales?page=21,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167,0.002093
4242,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4405,97.105.19.58,GET,/api/v1/sales?page=29,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4790,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4862,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4871,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
5520,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
5539,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
6860,97.105.19.58,GET,/api/v1/sales?page=115,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167,0.002093


In [51]:
df[df.size_mb == 0]

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_status_code_ip,proba_status_code_user
1745,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167,0.002093
1819,97.105.19.58,GET,/api/v1/sales?page=21,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167,0.002093
4242,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4405,97.105.19.58,GET,/api/v1/sales?page=29,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4790,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4862,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
4871,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
5520,97.105.19.58,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
5539,97.105.19.58,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0,0.0,0.001167,0.001000
6860,97.105.19.58,GET,/api/v1/sales?page=115,HTTP/1.1,499,0,python-requests/2.20.1,0.0,0.001167,0.002093


In [52]:
df[df.size_mb == 0] == df[(df.proba_status_code_user < .01) & (df.proba_status_code_ip < .01)]

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,proba_status_code_ip,proba_status_code_user
1745,True,True,True,True,True,True,True,True,True,True
1819,True,True,True,True,True,True,True,True,True,True
4242,True,True,True,True,True,True,True,True,True,True
4405,True,True,True,True,True,True,True,True,True,True
4790,True,True,True,True,True,True,True,True,True,True
4862,True,True,True,True,True,True,True,True,True,True
4871,True,True,True,True,True,True,True,True,True,True
5520,True,True,True,True,True,True,True,True,True,True
5539,True,True,True,True,True,True,True,True,True,True
6860,True,True,True,True,True,True,True,True,True,True


### The anomalies seem to be anywhere where the size is 0.

In [55]:
df[df.status_code == '200'].user_agent.value_counts(normalize=True)

python-requests/2.21.0                                                                                                        0.858954
python-requests/2.20.1                                                                                                        0.136605
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"    0.002436
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"                                           0.000573
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)"                                                                   0.000501
Slackbot 1.0 (+https://api.slack.com/robots)"                                                                                 0.000430
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"    0.000287
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWe